# 1.01 Grab b1515 songs from pickles
### JX, 03/28/2023
> Retrieve b1515 songs from Zeke's pickles, then save them as wav. 

In [1]:
from starling_rhythm.utils.paths import PROCESSED_DIR, ensure_dir

In [2]:
bID = 's_b1515_23'

In [3]:
## output directory
bID_DIR = PROCESSED_DIR / bID
bID_DIR

PosixPath('/mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1515_23')

## Find pickles of bout-checked b1515 data

In [4]:
from pathlib2 import Path

In [5]:
DATA_PATH = '/mnt/sphere/speech_bci/derived_data/' + bID
DATA_DIR = Path(DATA_PATH)
DATA_DIR

PosixPath('/mnt/sphere/speech_bci/derived_data/s_b1515_23')

In [6]:
## grab data from 9/20/22 - 9/29/22, when b1555 sings the most
pickle_paths = list(Path(DATA_DIR).rglob("2023-03-24/bouts_sglx/bout_checked.pickle"))

In [7]:
pickle_paths

[PosixPath('/mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-24/bouts_sglx/bout_checked.pickle')]

## View Pickle

In [8]:
import pandas as pd
import numpy as np

In [9]:
bout_checked = pd.read_pickle(pickle_paths[0])

In [10]:
bout_checked.keys()

Index(['start_ms', 'end_ms', 'start_sample', 'end_sample', 'p_step', 'rms_p',
       'peak_p', 'bout_check', 'file', 'len_ms', 'syl_in', 'n_syl', 'peaks_p',
       'n_peaks', 'l_p_ratio', 'waveform', 'confusing', 'valid_waveform',
       'valid', 'spectrogram', 'is_call'],
      dtype='object')

In [11]:
bouts = bout_checked

In [12]:
bouts = bouts[bouts['is_call'] != True] ## not calls
bouts = bouts[bouts['bout_check'] == True] ## bout checked
bouts = bouts[bouts['confusing'] != True] ## not confusing

In [13]:
len(bouts)

200

In [14]:
file_types = np.unique(bouts.file)
file_types

array(['/mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-24/sglx/0754_con_g0/wav_mic.wav',
       '/mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-24/sglx/1311_stim_g0/wav_mic.wav',
       '/mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-24/sglx/1547_g0/wav_mic.wav'],
      dtype=object)

In [15]:
conspecific = bouts[bouts.file == file_types[0]]
stimuli = bouts[bouts.file == file_types[1]]
alone = bouts[bouts.file == file_types[2]]

In [16]:
assert len(np.unique(conspecific.file)) == 1
assert len(np.unique(stimuli.file)) == 1
assert len(np.unique(alone.file)) == 1

In [17]:
np.unique(conspecific.file)

array(['/mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-24/sglx/0754_con_g0/wav_mic.wav'],
      dtype=object)

In [18]:
len(conspecific)

43

In [19]:
np.unique(stimuli.file)

array(['/mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-24/sglx/1311_stim_g0/wav_mic.wav'],
      dtype=object)

In [20]:
len(stimuli)

132

In [21]:
np.unique(alone.file)

array(['/mnt/sphere/speech_bci/derived_data/s_b1515_23/2023-03-24/sglx/1547_g0/wav_mic.wav'],
      dtype=object)

In [22]:
len(alone)

25

In [23]:
import pandas as pd
from tqdm.autonotebook import tqdm
import scipy.io.wavfile as wav

/tmp/ipykernel_27351/3870098420.py:2: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [24]:
str(pickle_paths[0]).split('/')

['',
 'mnt',
 'sphere',
 'speech_bci',
 'derived_data',
 's_b1515_23',
 '2023-03-24',
 'bouts_sglx',
 'bout_checked.pickle']

In [25]:
alone.file.values[0].split('/')[8]

'1547_g0'

In [26]:
rate = 48000

In [29]:
## For every pickle path
for pickle in tqdm(pickle_paths, desc = 'Processing each day'):
    
    ## load in the pickle
    bouts = pd.read_pickle(pickle)
    ## filter out calls
    bouts = bouts[bouts['is_call'] != True] ## not calls
    bouts = bouts[bouts['bout_check'] == True] ## yes, bout checked
    bouts = bouts[bouts['confusing'] != True] ## not confusing
    
    ## for each bout, find its audio, and construct its name
    for i, row in tqdm(bouts.iterrows(), desc = "Processing each day's bout"):
        data = row.waveform
        
        ## CONVENTION: bID_YYYY-MM-DD_TYPE_STRT_END.wav
        ymd = str(pickle).split('/')[6]
        song_type = row.file.split('/')[8]
        strt = str(row.start_ms)
        end = str(row.end_ms)
        
        filename = bID + '_' + ymd + '__' + song_type + '__' + strt + '_' + end + str('.wav')
        filename = PROCESSED_DIR / bID / 'raw' / filename
        
        ensure_dir(filename)
        
        print('Saving ' + str(filename))
        
        wav.write(filename = filename, rate = rate, data = data)

Processing each day:   0%|          | 0/1 [00:00<?, ?it/s]
Processing each day's bout: 0it [00:00, ?it/s]
Processing each day's bout: 1it [00:00,  3.17it/s]

Saving /mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1515_23/raw/s_b1515_23_2023-03-24__1311_stim_g0__1759420_1902930.wav
Saving /mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1515_23/raw/s_b1515_23_2023-03-24__1311_stim_g0__6883109_6988584.wav
Saving /mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1515_23/raw/s_b1515_23_2023-03-24__1311_stim_g0__7164904_7249199.wav



Processing each day's bout: 3it [00:00,  7.11it/s]
Processing each day's bout: 5it [00:00,  9.75it/s]

Saving /mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1515_23/raw/s_b1515_23_2023-03-24__1311_stim_g0__3799599_3865139.wav
Saving /mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1515_23/raw/s_b1515_23_2023-03-24__1311_stim_g0__5340034_5404354.wav
Saving /mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1515_23/raw/s_b1515_23_2023-03-24__1547_g0__13998691_14059066.wav
Saving /mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1515_23/raw/s_b1515_23_2023-03-24__1311_stim_g0__4427279_4487649.wav



Processing each day's bout: 7it [00:00, 11.96it/s]
Processing each day's bout: 9it [00:00, 13.12it/s]

...

Saving /mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1515_23/raw/s_b1515_23_2023-03-24__1311_stim_g0__4253664_4266219.wav
Saving /mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1515_23/raw/s_b1515_23_2023-03-24__1311_stim_g0__1929805_1941800.wav
Saving /mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1515_23/raw/s_b1515_23_2023-03-24__1311_stim_g0__672630_684315.wav
Saving /mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1515_23/raw/s_b1515_23_2023-03-24__1311_stim_g0__4120894_4132329.wav
Saving /mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1515_23/raw/s_b1515_23_2023-03-24__0754_con_g0__1347845_1358855.wav
Saving /mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1515_23/raw/s_b1515_23_2023-03-24__1311_stim_g0__1274420_1284925.wav
Saving /mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1515_23/raw/s_b1515_23_2023-03-24__1311_stim_g0__2070965_2081455.wav
Saving /mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1515_23/raw/s_b1515_23_2023-03-24__1311_stim_g0__7635119_7645339.wav
Saving /mnt/cube/j8


Processing each day's bout: 181it [00:06, 58.92it/s]
Processing each day's bout: 190it [00:06, 66.22it/s]

Saving /mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1515_23/raw/s_b1515_23_2023-03-24__0754_con_g0__5581331_5590161.wav
Saving /mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1515_23/raw/s_b1515_23_2023-03-24__1311_stim_g0__5495219_5503809.wav
Saving /mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1515_23/raw/s_b1515_23_2023-03-24__1547_g0__11688889_11697299.wav
Saving /mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1515_23/raw/s_b1515_23_2023-03-24__1547_g0__11703629_11711659.wav
Saving /mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1515_23/raw/s_b1515_23_2023-03-24__1547_g0__11667029_11674499.wav
Saving /mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1515_23/raw/s_b1515_23_2023-03-24__1311_stim_g0__5483424_5490799.wav
Saving /mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1515_23/raw/s_b1515_23_2023-03-24__1547_g0__15157406_15164571.wav
Saving /mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1515_23/raw/s_b1515_23_2023-03-24__0754_con_g0__4021243_4028278.wav
Saving /mnt/cube/j8xing/starli

Processing each day's bout: 200it [00:06, 29.96it/s]
Processing each day: 100%|██████████| 1/1 [00:14<00:00, 14.06s/it]

Saving /mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1515_23/raw/s_b1515_23_2023-03-24__1547_g0__20608280_20611880.wav
Saving /mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1515_23/raw/s_b1515_23_2023-03-24__0754_con_g0__2611018_2614333.wav
Saving /mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1515_23/raw/s_b1515_23_2023-03-24__1311_stim_g0__1331120_1334365.wav
Saving /mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1515_23/raw/s_b1515_23_2023-03-24__1311_stim_g0__2174085_2177110.wav
